In [110]:
import numpy as np
import pandas as pd
import re, os
import shutil
from tempfile import mkstemp
from astropy.table import Table, vstack, hstack, join
from bokeh.plotting import figure, show, output_file
from bokeh.models import Span, ColumnDataSource, Slider, CustomJS, Button, Range1d
from bokeh.io import output_notebook, curdoc
from bokeh.layouts import column, row
from bokeh.events import ButtonClick

# output_notebook()

In [111]:
## Read in data

spectrum = pd.read_csv('/Users/isabelkain/Desktop/GALEX/data/swp54894.csv')
wavelengths = spectrum['Wavelengths'].values
flux = spectrum['Flux'].values
fluxerr = spectrum['Fluxerr'].values


In [112]:
## Read in lines
lines = Table.read('/Users/isabelkain/Desktop/GALEX/data/swp54894_foundpeaks.ecsv')
lines

line_center,line_type,line_center_index,Peak label,Spectrum,Confident?
Angstrom,,,,,
float32,str8,int64,str4,str1,bool
1180.6892,emission,18,1181,?,False
1215.8912,emission,39,1216,?,False
1234.3304,emission,50,1234,?,False
1304.7346,emission,92,1305,?,False
1396.9305,emission,147,1397,?,False
1547.7964,emission,237,1548,?,False
1639.9923,emission,292,1640,?,False
1660.1079,emission,304,1660,?,False


In [113]:
## Plot

p = figure(title=f"Spectrum for * gam Mic (swp54894, G III-I, SNR = 3)", 
           y_axis_label=r"Flux (erg/cm^2/s/A)", 
           x_axis_label=r"Wavelength (Angstrom)", sizing_mode='stretch_width')

p.line(wavelengths, flux, line_width=2)
p.varea(x=wavelengths, y1=flux-fluxerr, y2=flux+fluxerr, fill_color="gray", alpha=0.5)


## Add slider for each identified peak

sliders = []

for i in range(len(lines)):
    
    slider = Slider(start=wavelengths[0], end=wavelengths[-1], 
                    value=lines['line_center'][i], 
                    step=0.01, title=lines['Peak label'][i], width=500)
    span = Span(location=slider.value, dimension='height', line_color="#B488D3", line_dash='dashed')
    p.add_layout(span)

    callback = CustomJS(args=dict(span=span), code="""
        span.location = cb_obj.value
    """)
    slider.js_on_change('value', callback)
    
    sliders.append(slider)
    
user_sliders = []

for i in range(5):
    
    slider = Slider(start=wavelengths[0], end=wavelengths[-1], 
                    value=wavelengths[-1], 
                    step=0.01, title=f'User line {i+1}', width=500)
    span = Span(location=slider.value, dimension='height', line_color="#2d8659", line_dash='dashed')
    p.add_layout(span)

    callback = CustomJS(args=dict(span=span), code="""
        span.location = cb_obj.value
    """)
    slider.js_on_change('value', callback)
    
    user_sliders.append(slider)
    
    
## Add button to save results
button = Button(label='Save inputs', button_type='success', align='center')
    
    
## Format and show

row2 = row(column(*sliders, sizing_mode='scale_width'), 
           column(*user_sliders, sizing_mode='scale_width'), sizing_mode='scale_width')   
layout = column(p, row2, button, sizing_mode='scale_width')
  
# layout = column(p, slider, sizing_mode='scale_width')
# curdoc().add_root(layout)
show(layout)


In [19]:
slider_values = [1180.6892, 1215.8912, 1234.3304, 1304.7346, 1396.9305, 
                 1547.7964, 1639.9923, 1660.1079, 1681.8995, 1817.679, 
                 1924.9614, 1951.782, 1970.2212]

for k in np.where(slider_values==wavelengths.max())[0]:
    slider_values[k] = 0.
    
slider_values

[1180.6892,
 1215.8912,
 1234.3304,
 1304.7346,
 1396.9305,
 1547.7964,
 1639.9923,
 1660.1079,
 1681.8995,
 1817.679,
 1924.9614,
 1951.782,
 1970.2212]

In [26]:
lines# = slider_values

line_center,line_type,line_center_index,Peak label,Spectrum,Confident?,Measured peak
Angstrom,,,,,,
float32,str8,int64,str4,str1,bool,float64
1180.6892,emission,18,1181,?,False,1180.6892
1215.8912,emission,39,1216,?,False,1215.8912
1234.3304,emission,50,1234,?,False,1234.3304
1304.7346,emission,92,1305,?,False,1304.7346
1396.9305,emission,147,1397,?,False,1396.9305
1547.7964,emission,237,1548,?,False,1547.7964
1639.9923,emission,292,1640,?,False,1639.9923
1660.1079,emission,304,1660,?,False,1660.1079


In [ ]:
testlabel = str(round(slider_values[0]))

In [29]:
user_slider_values = [1150.516, 1978.6027, 1978.6027, 1978.6027, 1978.6027]

for j in range(len(user_slider_values)):
    r = (0, 'emission', 0, str(round(user_slider_values[j])), '?', False, user_slider_values[j])
    lines.add_row(r)
    
lines

line_center,line_type,line_center_index,Peak label,Spectrum,Confident?,Measured peak
Angstrom,,,,,,
float32,str8,int64,str4,str1,bool,float64
1180.6892,emission,18,1181,?,False,1180.6892
1215.8912,emission,39,1216,?,False,1215.8912
1234.3304,emission,50,1234,?,False,1234.3304
1304.7346,emission,92,1305,?,False,1304.7346
1396.9305,emission,147,1397,?,False,1396.9305
1547.7964,emission,237,1548,?,False,1547.7964
1639.9923,emission,292,1640,?,False,1639.9923
1660.1079,emission,304,1660,?,False,1660.1079


In [30]:
testlines = Table.read('/Users/isabelkain/Desktop/GALEX/data/swp54894_foundpeaks_test.ecsv')
testlines

line_center,line_type,line_center_index,Peak label,Spectrum,Confident?,Measured
Angstrom,,,,,,
float32,str8,int64,str4,str1,bool,float64
1180.6892,emission,18,1181,?,False,0.0
1215.8912,emission,39,1216,?,False,1215.8912353515625
1234.3304,emission,50,1234,?,False,1234.3304443359375
1304.7346,emission,92,1305,?,False,1304.734619140625
1396.9305,emission,147,1397,?,False,1396.9305419921875
1547.7964,emission,237,1548,?,False,1547.79638671875
1639.9923,emission,292,1640,?,False,1639.9923095703125
1660.1079,emission,304,1660,?,False,1660.10791015625


In [57]:
swpid = 'TESTSWPID'

plotfile = '/Users/isabelkain/Desktop/GALEX/code/bokehplot.py'
search_text = 'swpid = \'*\'\n'
replace_text = f'swpid = \'{swpid}\'\n'

print(search_text, replace_text)

with open('/Users/isabelkain/Desktop/GALEX/code/bokehplot.py', 'r') as file:
    lines = file.readlines()
    
#     data = data.replace('swpid = \'', replace_text)
    test = re.sub(search_text, replace_text, data, count=0, flags=0)

swpid = '* swpid = 'TESTSWPID123'


In [75]:
# swpid = 'TESTSWPID'

# plotfile = '/Users/isabelkain/Desktop/GALEX/code/bokehplot.py'
# search_text = 'swpid = \'*\'\n'
# replace_text = f'swpid = \'{swpid}\'\n'


# with open(plotfile, 'r') as file:
    
#     lines = file.readlines()
    
#     for line in lines:
        
        
        
        
# #         newline = re.sub(search_text, replace_text, data, count=0, flags=0) #line.replace('*', f'swpid = \'{swpid}\'\n')
# #         display(line, newline)

'import pandas as pd\n'

In [88]:
# for i in range(len(lines)):

#     srch = re.search(search_text, lines[i])
#     if type(srch) != NoneType:
#         print('yes')

In [97]:
swpid = 'TEST123'
objname = 'teststar'
starname = 'test star'


with open('/Users/isabelkain/Desktop/GALEX/code/bokehplot.py') as f:
    content_list = f.readlines()

for i, line in enumerate(content_list):
    if 'swpid = ' in line:
        content_list[i] = f'swpid = \'{swpid}\''
        print(line, content_list[i])
    if 'objname = ' in line:
        content_list[i] = f'objname = \'{objname}\''
        print(line, content_list[i])
    if 'starname = ' in line:
        content_list[i] =f'starname = \'{starname}\''
        print(line, content_list[i])
        


swpid = 'swp54894'
 swpid = 'TEST123'
starname = '* gam Mic'
 starname = 'test star'
objname = '*gamMic'
 objname = 'teststar'


In [106]:
def replace_swpid(scriptpath, swpid, objname, starname):
    
    #Create temp file
    fh, abs_path = mkstemp()
    
    with os.fdopen(fh,'w') as new_file:
        
        with open(scriptpath) as old_file:
            
            content_list = old_file.readlines()
            
            for i, line in enumerate(content_list):
                if 'swpid = ' in line:
                    content_list[i] = f'swpid = \'{swpid}\'\n'
                    print(line, content_list[i])
                if 'objname = ' in line:
                    content_list[i] = f'objname = \'{objname}\'\n'
                    print(line, content_list[i])
                if 'starname = ' in line:
                    content_list[i] =f'starname = \'{starname}\'\n'
                    print(line, content_list[i])
            
                new_file.write(content_list[i])
            
                
    #Copy the file permissions from the old file to the new file
    shutil.copymode(scriptpath, abs_path)
    
    #Remove original file
    os.remove(scriptpath)
    
    #Move new file
    shutil.move(abs_path, scriptpath)

In [107]:
swpid = 'TESTSWPID'
objname = 'TESTOBJNAME'
starname = 'TESTSTARNAME'

scriptpath = '/Users/isabelkain/Desktop/GALEX/code/bokehplot.py'

replace_swpid(scriptpath, swpid, objname, starname)

swpid = 'xyz'
 swpid = 'TESTSWPID'

starname = 'abc'
 starname = 'TESTSTARNAME'

objname = '123'
 objname = 'TESTOBJNAME'



In [ ]:
import shutil
from tempfile import mkstemp